# StrategicGrowthAnalysis-UniqueGiftsLtd

=== Phase 1: Data Sanitation and Preprocessing ===


Download Dataset from Google Drive

In [ ]:
import requests # for downloading the dataset
import os # for file operations
from tqdm import tqdm  # progress bar library

url = "https://docs.google.com/spreadsheets/d/1RZ0nYqAUgSivbfIiMsbZfEy2yg5KzNBcbGThzDPHx7c/export?format=csv"
file_path = "../data/dataset.csv" # Path to save the dataset

os.makedirs("../data", exist_ok=True)

response = requests.get(url, stream=True)
total_size = int(response.headers.get('content-length', 0))
block_size = 1024  # 1 KB

print("Downloading dataset with progress:")

with open(file_path, "wb") as file, tqdm(
    desc=file_path,
    total=total_size,
    unit='iB',
    unit_scale=True,
    unit_divisor=1024,
) as bar:
    for data in response.iter_content(block_size):
        file.write(data)
        bar.update(len(data))

if os.path.exists(file_path):
    print("Download complete!")
else:
    print("Download failed. Please check the URL or your internet connection.")




Check if Dataset File Exists

In [ ]:
import pandas as pd

if not os.path.exists(file_path): # Check if the file exists
    raise FileNotFoundError(f"Dataset file not found at {file_path}. Please check the download process.")

df = pd.read_csv(file_path) # Load the dataset
print(f"Dataset loaded successfully with {df.shape[0]} rows and {df.shape[1]} columns.")


Load Dataset into Pandas DataFrame

In [ ]:
# Display the first few rows of the dataset
try:
    df = pd.read_csv('../data/dataset.csv')  # Adjust the path as necessary
    print("Dataset loaded successfully!")
except FileNotFoundError:
    print("Error: The file 'online_retail.csv' was not found in the 'data' folder.")
    print("Please make sure you have downloaded the dataset and placed it in the correct folder.")

# Display the first 5 rows to get an idea of the data
df.head()

Initial Data Audit (.info(), .describe(), Missing Values)

In [ ]:
print("Dataframe Info:")
df.info()

print("\n" + "="*50 + "\n")

# Get summary statistics for numerical columns
print("Numerical Describe:")
print(df.describe())
print("\n" + "="*50 + "\n")

# Count missing values in each column
print("Missing Values Count:")
print(df.isnull().sum())

Identify and Remove Duplicate Rows

In [ ]:
# Print shape before dropping duplicates
print(f"Shape before dropping duplicates: {df.shape}")

print(df.duplicated().sum())

# Drop duplicate rows
df.drop_duplicates(inplace=True) # re df

# Print shape after dropping duplicates
print(f"Shape after dropping duplicates: {df.shape}")

# Drop rows where Customer ID is missing
df.dropna(subset=['Customer ID'], inplace=True)

# Print shape after dropping missing Customer ID
print(f"Shape after dropping duplicates: {df.shape}")

# Verify that missing Customer IDs are handled
print("\nMissing values after handling Customer ID:")
print(df.isnull().sum())

Remove Cancelled Orders and Invalid Quantities

In [ ]:
# Remove cancelled orders (Invoice starts with 'C')
df = df[~df['Invoice'].astype(str).str.startswith('C')]

# Ensure quantity is positive
df = df[df['Quantity'] > 0]

# Remove records where price is 0
df = df[df['Price'] > 0]

# Check the shape of the dataframe after cleaning
print(f"Shape after cleaning transactions: {df.shape}")


Filter Non-Product Stock Codes (POST, M, etc.)

In [ ]:
# (Assuming 'df' is your dataframe after the initial cleaning)

def is_product_code(code):

    code_str = str(code)
    return any(char.isdigit() for char in code_str)

# --- Identify codes that would be removed using this new logic ---
all_unique_codes = df['StockCode'].unique()
codes_to_be_removed = [code for code in all_unique_codes if not is_product_code(code)]
print(f"Following codes (without any digits) will be removed: {codes_to_be_removed}")


# --- Filtering Step ---
print(f"\nShape before filtering: {df.shape}")

# Apply the function to the 'StockCode' column and keep only the rows that return True
df = df[df['StockCode'].apply(is_product_code)]

print(f"Shape after filtering: {df.shape}")


Create TotalPrice Feature (Quantity × Price)

In [ ]:
# Create TotalPrice column
df['TotalPrice'] = df['Quantity'] * df['Price']

print(df.shape)
print(df.head())

Convert InvoiceDate to Datetime Format

In [ ]:
# Convert InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

print(type(df['InvoiceDate']))
print(df['InvoiceDate'].dtype)

# 2. create new columns  - Year, Month, DayOfWeek, HourOfDay
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek     # Monday=0, Sunday=6
df['HourOfDay'] = df['InvoiceDate'].dt.hour

# 3. view the new columns (first 5 rows)
print(df[['InvoiceDate', 'Year', 'Month', 'DayOfWeek', 'HourOfDay']].head())


Convert Customer ID to Integer Type

In [ ]:
df['Customer ID'] = df['Customer ID'].astype(int)  # Convert Customer ID to integer

print(df['Customer ID'].dtype)

Convert StockCode to String Type

In [ ]:
# Convert StockCode to string
df['StockCode'] = df['StockCode'].astype(str)  # Convert StockCode to string
print(df['StockCode'].dtype)

Save Cleaned Dataset to CSV

In [ ]:
df.to_csv('../data/cleaned-dataset.csv', index=False)

=== Phase 2: Exploratory Data Analysis (EDA) ====

Set Up Visualization Libraries (Matplotlib, Seaborn)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Visualization style
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

# Dataset load (Cleaned dataset)
import pandas as pd

# Robustly load and standardize the cleaned dataset (handles varying column names and ensures dates are parsed)
print("Loading cleaned dataset (robust)...")
df = pd.read_csv('../data/cleaned-dataset.csv', low_memory=False)

# --- Find and parse invoice date column ---
date_col = None
for c in df.columns:
    if 'invoice' in c.lower() and 'date' in c.lower():
        date_col = c
        break
if not date_col:
    # fallback common name
    if 'InvoiceDate' in df.columns:
        date_col = 'InvoiceDate'

if not date_col:
    raise ValueError(f"No invoice date column found in cleaned-dataset.csv. Columns: {df.columns.tolist()}")

df[date_col] = pd.to_datetime(df[date_col], errors='coerce', dayfirst=True)
df.rename(columns={date_col: 'InvoiceDate'}, inplace=True)

# --- Standardize important column names (handles variants from different dataset exports) ---
rename_map = {}
for c in df.columns:
    lc = c.lower().replace(' ', '').replace('_', '')
    if lc in ('invoiceno', 'invoice'):
        rename_map[c] = 'Invoice'
    if lc in ('invoicedate',):
        rename_map[c] = 'InvoiceDate'
    if lc in ('unitprice', 'price', 'unitpriceinc'):
        rename_map[c] = 'Price'
    if lc in ('customerid', 'customerid.'):
        rename_map[c] = 'Customer ID'
    if lc in ('stockcode',):
        rename_map[c] = 'StockCode'
    if lc in ('quantity',):
        rename_map[c] = 'Quantity'
    if lc in ('description',):
        rename_map[c] = 'Description'
    if lc in ('country',):
        rename_map[c] = 'Country'

df.rename(columns=rename_map, inplace=True)

# Verify required columns exist
required = ['Invoice', 'InvoiceDate', 'Price', 'Customer ID', 'Quantity']
missing = [r for r in required if r not in df.columns]
if missing:
    raise ValueError(f"Missing required columns in cleaned-dataset.csv: {missing}. Available: {df.columns.tolist()}")

# Ensure numeric types
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce').fillna(0).astype(int)
df['Price'] = pd.to_numeric(df['Price'], errors='coerce').fillna(0.0)

# Ensure TotalPrice exists
if 'TotalPrice' not in df.columns:
    df['TotalPrice'] = df['Quantity'] * df['Price']

print(f"Dataset loaded and standardized. Shape: {df.shape}")

Monthly Sales Trend Analysis

In [ ]:
# Year-Month column
df['YearMonth'] = df['InvoiceDate'].dt.to_period('M')

# Group by Year-Month and sum TotalPrice
monthly_sales = df.groupby('YearMonth')['TotalPrice'].sum().reset_index()
monthly_sales['YearMonth'] = monthly_sales['YearMonth'].astype(str)  # Convert YearMonth to string for plotting

# Line chart Plot
plt.figure(figsize=(12, 6))
sns.lineplot(x='YearMonth', y='TotalPrice', data=monthly_sales, marker="o")
plt.title("Monthly Sales Trend")
plt.xlabel("Month")
plt.ylabel("Total Sales Revenue (£)")
plt.xticks(rotation=45)
plt.show()


Extract Year and Month from InvoiceDate

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Extract Year and Month from InvoiceDate
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['MonthName'] = df['InvoiceDate'].dt.strftime('%B')

# Group by Year and MonthName to get total sales
monthly_sales_yearly = df.groupby(['Year', 'MonthName'])['TotalPrice'].sum().reset_index()

# Reorder the months for better visualization
month_order = ['January','February','March','April','May','June',
               'July','August','September','October','November','December']
monthly_sales_yearly['MonthName'] = pd.Categorical(monthly_sales_yearly['MonthName'], 
                                                   categories=month_order, 
                                                   ordered=True)
monthly_sales_yearly = monthly_sales_yearly.sort_values(['Year', 'MonthName'])

# Plotting the monthly sales by year
plt.figure(figsize=(12,6))
sns.lineplot(data=monthly_sales_yearly, x='MonthName', y='TotalPrice', hue='Year', marker='o')
plt.title("Monthly Sales by Year (Check for Feb & Apr Dips)")
plt.xlabel("Month")
plt.ylabel("Total Sales Revenue (£)")
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

# February data
feb_sales = monthly_sales_yearly[monthly_sales_yearly['MonthName'] == "February"]['TotalPrice']
# April data
apr_sales = monthly_sales_yearly[monthly_sales_yearly['MonthName'] == "April"]['TotalPrice']

# Mean & std check
feb_mean = feb_sales.mean()
apr_mean = apr_sales.mean()
overall_mean = monthly_sales_yearly['TotalPrice'].mean()

print(f"February Mean Sales: {feb_mean}")
print(f"April Mean Sales: {apr_mean}")
print(f"Overall Mean Sales: {overall_mean}")

# Check pattern: True if all years' Feb sales < overall mean
if hasattr(feb_sales, 'empty') and feb_sales.empty:
    feb_pattern = False
else:
    feb_pattern = bool((feb_sales < overall_mean).all())

if hasattr(apr_sales, 'empty') and apr_sales.empty:
    apr_pattern = False
else:
    apr_pattern = bool((apr_sales < overall_mean).all())

print(f"February consistent dip? {feb_pattern}")
print(f"April consistent dip? {apr_pattern}")

Sales by Day of Week and Hour of Day

In [ ]:
df['DayOfWeek'] = df['InvoiceDate'].dt.day_name()
df['HourOfDay'] = df['InvoiceDate'].dt.hour

# Sales by Day
sales_by_day = df.groupby('DayOfWeek')['TotalPrice'].sum().reindex(
    ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
)

# Bar chart – Day
sales_by_day.plot(kind='bar', color='skyblue')
plt.title("Sales by Day of Week")
plt.ylabel("Total Sales Revenue (£)")
plt.show()

# Sales by Hour
sales_by_hour = df.groupby('HourOfDay')['TotalPrice'].sum()
sales_by_hour.plot(kind='bar', color='lightgreen')
plt.title("Sales by Hour of Day")
plt.ylabel("Total Sales Revenue (£)")
plt.show()


Top 10 Countries by Total Revenue

In [ ]:
country_sales = df.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False).reset_index()

# Top 10
top_10_countries = country_sales.head(10)

# Plot
sns.barplot(x='TotalPrice', y='Country', data=top_10_countries, palette="viridis")
plt.title("Top 10 Countries by Sales Revenue")
plt.xlabel("Total Sales Revenue (£)")
plt.ylabel("Country")
plt.show()

UK vs Rest of World Revenue Share

In [ ]:
uk_revenue = df[df['Country'] == 'United Kingdom']['TotalPrice'].sum()
total_revenue = df['TotalPrice'].sum()
uk_percentage = (uk_revenue / total_revenue) * 100
print(f"UK Revenue %: {uk_percentage:.2f}%")

Revenue Contribution by Country (Chart)

In [ ]:
# Calculate revenue by country
country_revenue = df.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False)

# Calculate percentages
country_percentages = (country_revenue / total_revenue) * 100

# For better visualization, group smaller countries (< 1%) into "Others"
threshold = 1.0
major_countries = country_percentages[country_percentages >= threshold]
minor_countries_sum = country_percentages[country_percentages < threshold].sum()

# Create data for pie chart
if minor_countries_sum > 0:
    pie_data = major_countries.copy()
    pie_data['Others'] = minor_countries_sum
else:
    pie_data = major_countries

# Create pie chart
plt.figure(figsize=(12, 8))
colors = plt.cm.Set3(range(len(pie_data)))
wedges, texts, autotexts = plt.pie(pie_data.values,
                                   labels=pie_data.index,
                                   autopct='%1.1f%%',
                                   colors=colors,
                                   startangle=90)

plt.title('Revenue Distribution by Country', fontsize=16, fontweight='bold')

# Improve text readability
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

plt.axis('equal')
plt.tight_layout()
plt.show()

# Print detailed breakdown
print("\nRevenue Breakdown by Country:")
print("-" * 40)
for country, percentage in country_percentages.head(10).items():
    revenue = country_revenue[country]
    print(f"{country}: £{revenue:,.2f} ({percentage:.2f}%)")

if minor_countries_sum > 0:
    print(f"\nOthers (countries < 1%): £{country_revenue[country_percentages < threshold].sum():,.2f} ({minor_countries_sum:.2f}%)")

Revenue by Day of Week (Bar Chart)

In [ ]:
import matplotlib.pyplot as plt

# Revenue by DayOfWeek
day_sales = df.groupby('DayOfWeek')['TotalPrice'].sum()
day_sales.plot(kind='bar', figsize=(7,4), title="Revenue by Day of Week")
plt.ylabel("Revenue")
plt.show()

# Revenue by HourOfDay
hour_sales = df.groupby('HourOfDay')['TotalPrice'].sum()
hour_sales.plot(kind='bar', figsize=(7,4), title="Revenue by Hour of Day")
plt.ylabel("Revenue")
plt.show()

Top 10 Products by Quantity and Revenue

In [ ]:
# Top 10 by Quantity Sold
top_quantity = df.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10)

# Top 10 by Revenue
top_revenue = df.groupby('Description')['TotalPrice'].sum().sort_values(ascending=False).head(10)

print("Top 10 by Quantity Sold:\n", top_quantity)
print("\nTop 10 by Revenue:\n", top_revenue)

Visualize Top Products: Quantity vs Revenue

In [ ]:
# Create subplots for both charts
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Top 10 by Quantity - Bar Chart
top_quantity.plot(kind='barh', ax=ax1, color='skyblue')
ax1.set_title('Top 10 Products by Quantity Sold', fontsize=14, fontweight='bold')
ax1.set_xlabel('Total Quantity Sold')
ax1.set_ylabel('Product Description')

# Top 10 by Revenue - Bar Chart  
top_revenue.plot(kind='barh', ax=ax2, color='lightgreen')
ax2.set_title('Top 10 Products by Revenue', fontsize=14, fontweight='bold')
ax2.set_xlabel('Total Revenue (£)')
ax2.set_ylabel('Product Description')

plt.tight_layout()
plt.show()

# Print summary statistics
print(f"\nTop quantity product: {top_quantity.index[0]} ({top_quantity.iloc[0]:,} units)")
print(f"Top revenue product: {top_revenue.index[0]} (£{top_revenue.iloc[0]:,.2f})")

=== Phase 3: Advanced Analytics – RFM Segmentation ===

Prepare Data for RFM Analysis, Scoring, and Wholesaler Classification

In [32]:
import pandas as pd
import numpy as np

# Use the already-loaded `df` (from earlier EDA). If not present, load robustly.
try:
    df
except NameError:
    df = pd.read_csv('../data/cleaned-dataset.csv', low_memory=False)

# Ensure InvoiceDate is datetime
if df['InvoiceDate'].dtype == object or not np.issubdtype(df['InvoiceDate'].dtype, np.datetime64):
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce', dayfirst=True)

# Ensure TotalPrice exists
if 'TotalPrice' not in df.columns:
    df['TotalPrice'] = pd.to_numeric(df['Quantity'], errors='coerce').fillna(0) * pd.to_numeric(df['Price'], errors='coerce').fillna(0.0)

# Snapshot date = 1 day after last InvoiceDate
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)
print(f"Snapshot date: {snapshot_date}")

# Recency
recency_df = df.groupby('Customer ID').agg(Recency=('InvoiceDate', lambda x: (snapshot_date - x.max()).days)).reset_index()

# Frequency (number of unique invoices)
frequency_df = df.groupby('Customer ID').agg(Frequency=('Invoice', 'nunique')).reset_index()

# Monetary (total spent)
monetary_df = df.groupby('Customer ID').agg(Monetary=('TotalPrice', 'sum')).reset_index()

# Merge
rfm_df = recency_df.merge(frequency_df, on='Customer ID').merge(monetary_df, on='Customer ID')

# Add average order value and average order quantity to help classify wholesalers
# avg_order_value = Monetary / Frequency
rfm_df['AvgOrderValue'] = rfm_df['Monetary'] / rfm_df['Frequency']

# average quantity per invoice per customer
invoice_qty = df.groupby(['Customer ID', 'Invoice']).agg(InvoiceQty=('Quantity', 'sum')).reset_index()
avg_qty = invoice_qty.groupby('Customer ID').agg(AvgOrderQty=('InvoiceQty', 'mean')).reset_index()
rfm_df = rfm_df.merge(avg_qty, on='Customer ID', how='left')

# RFM scoring using quintiles. Use rank for frequency to reduce qcut issues on identical values.
# Handle small number of customers by using qcut with duplicates dropped when necessary.
r_labels = [5,4,3,2,1]
f_labels = [1,2,3,4,5]
m_labels = [1,2,3,4,5]

try:
    rfm_df['R_Score'] = pd.qcut(rfm_df['Recency'], 5, labels=r_labels)
    rfm_df['F_Score'] = pd.qcut(rfm_df['Frequency'].rank(method='first'), 5, labels=f_labels)
    rfm_df['M_Score'] = pd.qcut(rfm_df['Monetary'], 5, labels=m_labels)
except ValueError:
    # fallback when there are too few unique values
    rfm_df['R_Score'] = pd.cut(rfm_df['Recency'], bins=5, labels=r_labels)
    rfm_df['F_Score'] = pd.cut(rfm_df['Frequency'].rank(method='first'), bins=5, labels=f_labels)
    rfm_df['M_Score'] = pd.cut(rfm_df['Monetary'], bins=5, labels=m_labels)

# Convert to int (coerce if NaN)
rfm_df['R_Score'] = pd.to_numeric(rfm_df['R_Score'], errors='coerce').fillna(1).astype(int)
rfm_df['F_Score'] = pd.to_numeric(rfm_df['F_Score'], errors='coerce').fillna(1).astype(int)
rfm_df['M_Score'] = pd.to_numeric(rfm_df['M_Score'], errors='coerce').fillna(1).astype(int)

# RFM segment code and description
rfm_df['RFM_Segment'] = rfm_df['R_Score'].astype(str) + rfm_df['F_Score'].astype(str) + rfm_df['M_Score'].astype(str)

def map_rfm_segment(row):
    r, f, m = row['R_Score'], row['F_Score'], row['M_Score']
    if r >= 4 and f >= 4 and m >= 4:
        return 'Champions'
    if f >= 4 and r >= 3:
        return 'Loyal Customers'
    if r >= 4 and f <= 2:
        return 'Recent Low-Frequency'
    if r <= 2 and f <= 2:
        return 'Hibernating'
    if r <= 2 and f >= 3:
        return 'At-Risk Customers'
    return 'Potential Loyalists'

rfm_df['RFM_Description'] = rfm_df.apply(map_rfm_segment, axis=1)

# Classify Wholesaler vs Retail based on simple heuristics
# Thresholds can be tuned; these are initial suggestions per project spec
wholesale_qty_threshold = 20  # avg qty per order
wholesale_value_threshold = 200.0  # avg order value

rfm_df['CustomerType'] = np.where((rfm_df['AvgOrderQty'] > wholesale_qty_threshold) | (rfm_df['AvgOrderValue'] > wholesale_value_threshold), 'Wholesaler', 'Retail')

# Summary comparisons
summary = rfm_df.groupby('CustomerType').agg(
    Customers=('Customer ID', 'nunique'),
    AvgFrequency=('Frequency', 'mean'),
    AvgMonetary=('Monetary', 'mean'),
    AvgOrderQty=('AvgOrderQty', 'mean'),
    AvgOrderValue=('AvgOrderValue', 'mean')
).reset_index()

print('\nCustomer type summary:')
print(summary)

# Save RFM segments
rfm_df.to_csv('../data/rfm_segments.csv', index=False)
print('\nSaved RFM segments to ../data/rfm_segments.csv')

Snapshot date: 2011-12-11 17:19:00

Customer type summary:
  CustomerType  Customers  AvgFrequency  AvgMonetary  AvgOrderQty  \
0       Retail        125      2.720000   140.627840    11.399038   
1   Wholesaler       5728      6.332577  2979.138843   256.747940   

   AvgOrderValue  
0      61.894360  
1     388.211929  

Saved RFM segments to ../data/rfm_segments.csv


Define RFM Scoring and Segment Mapping

In [ ]:
# Define segment mapping
def rfm_segment(row):
    r, f, m = row['R_Score'], row['F_Score'], row['M_Score']
    if r >= 4 and f >= 4 and m >= 4:
        return 'Champions'
    elif r >= 4 and f >= 4:
        return 'Loyal Customers'
    elif r >= 4 and f <= 2:
        return 'New Customers'
    elif r <= 2 and f >= 3:
        return 'At-Risk Customers'
    else:
        return 'Hibernating'

rfm_df['Segment'] = rfm_df.apply(rfm_segment, axis=1)
rfm_df['Segment'].value_counts()

create and save country_codes_enrichment.csv

In [ ]:
import requests
import time

countries = df['Country'].dropna().unique().tolist()
country_rows = []
for country in countries:
    try:
        resp = requests.get(f'https://restcountries.com/v3.1/name/{country}?fullText=true', timeout=5)
        if resp.status_code == 200:
            data = resp.json()
            if isinstance(data, list) and len(data) > 0:
                c = data[0]
                iso2 = c.get('cca2')
                iso3 = c.get('cca3')
                country_rows.append({'Country': country, 'ISO2': iso2, 'ISO3': iso3})
        else:
            country_rows.append({'Country': country, 'ISO2': None, 'ISO3': None})
    except Exception:
        country_rows.append({'Country': country, 'ISO2': None, 'ISO3': None})
    time.sleep(0.15)

country_codes = pd.DataFrame(country_rows)
country_codes.to_csv('../data/country_codes_enrichment.csv', index=False)
print('\nSaved country enrichment to ../data/country_codes_enrichment.csv')

=== Phase 4: Strategic Recommendations ===

Customer Monetary Value Distribution (Wholesaler Hypothesis Check)

In [ ]:
# Distribution of Monetary value per customer
rfm_df['Monetary'].plot(kind='hist', bins=50, figsize=(8,5), title="Customer Monetary Distribution")
plt.xlabel("Monetary Value")
plt.show()

rfm_df['Monetary'].plot(kind='box', figsize=(6,4), title="Monetary Value per Customer")
plt.show()

=== Phase 5: Data Enrichment via API Integration ===

Fetch Exchange Rates from API (GBP → USD, EUR) ,
Apply Exchange Rates to Top 100 Transactions

In [ ]:
import requests

# Use a free API that doesn't require authentication
url = "https://api.fxratesapi.com/latest?base=GBP&currencies=USD,EUR"

try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()

    if 'rates' in data:
        gbp_to_usd = data['rates']['USD']
        gbp_to_eur = data['rates']['EUR']
        print(f"Live rates - GBP to USD: {gbp_to_usd}, GBP to EUR: {gbp_to_eur}")
    else:
        # Fallback rates
        gbp_to_usd = 1.27
        gbp_to_eur = 1.17
        print("Using fallback rates")

except requests.RequestException as e:
    print(f"API request failed: {e}")
    # Fallback rates
    gbp_to_usd = 1.27
    gbp_to_eur = 1.17
    print("Using fallback rates")

# Apply currency conversion
df['TotalPrice_USD'] = df['TotalPrice'] * gbp_to_usd
df['TotalPrice_EUR'] = df['TotalPrice'] * gbp_to_eur

# Display results
print(df[['Invoice', 'TotalPrice', 'TotalPrice_USD', 'TotalPrice_EUR']].head(100))